In [3]:
import json
from requests_oauthlib import OAuth2Session
import requests
import pandas as pd
from get_data import get_campaigns, get_adsquads

# 0 - Import your credentials

In [17]:
with open("snapchat_credentials.json", "r") as f:
    snap_credentials = json.load(f)
snap_credentials

{'client_id': 'a8aaf469-e9d3-47db-a076-c2eab2c64ffc',
 'client_secret': '425b78a022260dbd31a1',
 'redirect_url': 'https://www.paloceras.com',
 'ad_account_id': 'c4d11bf0-981f-4365-a07b-90bc3c9c900e'}

# 1 - Generate access token from client_ID and client_secret

In [18]:
scope = ["snapchat-marketing-api"]
authorize_url = "https://accounts.snapchat.com/login/oauth2/authorize"
access_token_url = "https://accounts.snapchat.com/login/oauth2/access_token"
protected_url = "https://adsapi.snapchat.com/v1/me/organizations"

In [19]:
oauth = OAuth2Session(
    snap_credentials["client_id"],
    redirect_uri=snap_credentials["redirect_url"],
    scope=scope,
)

In [20]:
authorization_url, state = oauth.authorization_url(authorize_url)
print("Please go to %s and authorize access." % authorization_url)

Please go to https://accounts.snapchat.com/login/oauth2/authorize?response_type=code&client_id=a8aaf469-e9d3-47db-a076-c2eab2c64ffc&redirect_uri=https%3A%2F%2Fwww.paloceras.com&scope=snapchat-marketing-api&state=WcawDUsMtPYJZG5pN6TvSzHtoCq3bm and authorize access.


In [21]:
authorization_response = input("Enter the full callback URL: ")
# input a full link after click on "Continue" button in authorization_url

In [22]:
token = oauth.fetch_token(
    access_token_url,
    authorization_response=authorization_response,
    client_secret=snap_credentials["client_secret"],
    scope=scope,
)

In [23]:
snap_credentials["access_token"] = oauth.token["access_token"]
snap_credentials["refresh_token"] = oauth.token["refresh_token"]

# 2 - Generate access token from refresh_token

In [24]:
access_params = {
    "client_id": snap_credentials["client_id"],
    "client_secret": snap_credentials["client_secret"],
    "code": snap_credentials["refresh_token"],  # Get it in first step in redirect URL
    "grant_type": "refresh_token",
}

In [25]:
res = requests.post(access_token_url, params=access_params)

In [26]:
snap_credentials["access_token"] = res.json()["access_token"]
snap_credentials["refresh_token"] = res.json()["refresh_token"]
snap_credentials

{'client_id': 'a8aaf469-e9d3-47db-a076-c2eab2c64ffc',
 'client_secret': '425b78a022260dbd31a1',
 'redirect_url': 'https://www.paloceras.com',
 'ad_account_id': 'c4d11bf0-981f-4365-a07b-90bc3c9c900e',
 'access_token': 'eyJpc3MiOiJodHRwczpcL1wvYWNjb3VudHMuc25hcGNoYXQuY29tXC9hY2NvdW50c1wvb2F1dGgyXC90b2tlbiIsInR5cCI6IkpXVCIsImVuYyI6IkExMjhDQkMtSFMyNTYiLCJhbGciOiJkaXIiLCJraWQiOiJhY2Nlc3MtdG9rZW4tYTEyOGNiYy1oczI1Ni4wIn0..UWxR-BUDdN3Gv5EP3aPLuA.dLygzoQ9s5GNWHL80mjUI9f_-e7O7mo_fBOjB8_hIxrj-PG0Ly65f9cgm3lXgj78CQcjw68hEjoswyk3N0aq90SkWAIiYLB17WbnouFFkFTEsnlhI2JLGsXQUc9ihzOBI7BXisrmThhKZw4dstgiSxv0O2vpMrTtSdGxgaIm6hExu4GwUIsPd1F4NdD60xsLI9q3ytaG5ejQb9ASpCSf_fN4EvaZljIk0jUhcDzC4zTjMeNRuNshU5ZZkTmxXvxYkJoE7NHQnWkcPjS0IMoH4I_msXJYruZMjY6mmPN813oZjgidFLZApbK9YgDktcfYOLJCQfO6yakp1nrXlRS9pFKE4U_MBXDDNDVikPvYtoFT78IxyeF7aoNpXDu0HRtVJUBvaA70ZXtTkL1zYJH8W03Qj3YH3DaERVDuFtCxQcAtXDG3EAiMeCo-eP6mB1yJrDEwqJ-tIrfpSe4DfisujRU13RBBvEdKo9fvijRm1vBWmtQa1L0a0PRcro_onKfpYXvqcFYeqEg_shZZjkbkVxR0fVvfPVmFYm6HI0PhM_qZsn

# 3 - Get Snapchat Ads data

In [29]:
adsquad_metrics = get_adsquads(
    snap_credentials["ad_account_id"], snap_credentials["access_token"]
)
adsquads = adsquad_metrics["adsquads"]

campaign_metrics = get_campaigns(
    snap_credentials["ad_account_id"], snap_credentials["access_token"]
)
campaigns = campaign_metrics["campaigns"]

data = []

for cam in campaigns:
    campaign_id = cam["campaign"]["id"]
    campaign_name = cam["campaign"]["name"]
    time = cam["campaign"]["start_time"]

    matching_adsquads = [
        adsquad
        for adsquad in adsquads
        if adsquad["adsquad"]["campaign_id"] == campaign_id
    ]

    for adsquad in matching_adsquads:
        adsquad_name = adsquad["adsquad"]["name"]

        if not "total_stats" in cam:
            cam["total_stats"] = [
                {
                    "total_stat": {
                        "stats": {
                            "impressions": 0,
                            "swipes": 0,
                            "spend": 0,
                            "paid_impressions": 0,
                            "video_views": 0,
                            "conversion_ad_click": 0,
                            "conversion_rate": 0,
                        }
                    }
                }
            ]

        total_stat = cam["total_stats"][0]["total_stat"]["stats"]

        # Append data to list
        data.append(
            {
                "Campaign ID": campaign_id,
                "Campaign Name": campaign_name,
                "Date": time,
                "Impressions": total_stat["impressions"],
                "Swipes": total_stat["swipes"],
                "Amount Spent": total_stat["spend"],
                "Paid Impressions": total_stat["paid_impressions"],
                "Click Rate": total_stat["conversion_rate"],
                "Adset Name": adsquad_name,
            }
        )

,Campaign ID,Campaign Name,Date,Impressions,Swipes,Amount Spent,Paid Impressions,Click Rate,Adset Name
0,13ebf7c9-d80f-4743-aa82-00be44d40f6c,Fake Data,2024-04-02T02:05:49.791Z,0,0,0,0,0,"Finland, All Genders, All Ages"
1,c8bdafad-3771-4dd4-b399-5b79b92875fe,Website Conversions,2024-04-01T14:51:58.783Z,0,0,0,0,0,"Finland, All Genders, All Ages - Test"
2,c8bdafad-3771-4dd4-b399-5b79b92875fe,Website Conversions,2024-04-01T14:51:58.783Z,0,0,0,0,0,"Finland, All Genders, All Ages - Test2"


# 4 - Display the data in table

In [ ]:
df = pd.DataFrame(data)
df